In [4]:
%pip install geopandas
%pip install rasterio
%pip install shapely

                                              0.0/1.1 MB ? eta -:--:--
     -------                                  0.2/1.1 MB 4.4 MB/s eta 0:00:01
     ---------------------------------------  1.1/1.1 MB 11.7 MB/s eta 0:00:01
     ---------------------------------------- 1.1/1.1 MB 8.8 MB/s eta 0:00:00
                                              0.0/22.9 MB ? eta -:--:--
     ---                                      2.1/22.9 MB 44.2 MB/s eta 0:00:01
     -----                                    3.0/22.9 MB 32.0 MB/s eta 0:00:01
     -------                                  4.2/22.9 MB 33.4 MB/s eta 0:00:01
     -------                                  4.5/22.9 MB 23.9 MB/s eta 0:00:01
     ----------                               5.8/22.9 MB 24.9 MB/s eta 0:00:01
     -------------                            7.6/22.9 MB 27.1 MB/s eta 0:00:01
     ----------------                         9.5/22.9 MB 28.8 MB/s eta 0:00:01
     ------------------                      10.8/22.9 MB 28.5

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/24.3 MB ? eta -:--:--
                                              0.1/24.3 MB 1.3 MB/s eta 0:00:19
                                              0.3/24.3 MB 4.2 MB/s eta 0:00:06
     -                                        0.9/24.3 MB 7.5 MB/s eta 0:00:04
     ---                                      2.3/24.3 MB 13.5 MB/s eta 0:00:02
     -------                                  4.3/24.3 MB 19.8 MB/s eta 0:00:02
     ----------                               6.2/24.3 MB 23.4 MB/s eta 0:00:01
     -------------                            8.4/24.3 MB 26.8 MB/s eta 0:00:01
     -----------------------                 14.7/24.3 MB 81.8 MB/s eta 0:00:01
     --------------------------              16.7/24.3 MB 93.9 MB/s eta 0:00:01
     ---------------------------             17.3/24.3 MB 65.2 MB/s eta 0:00:01
     ------------------------------------    23.1/24.3 MB 72.6 MB/s eta 0:00:01
     --------------------------------------  24.3/2

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import os
from pprint import pprint
import numpy as np
from matplotlib import pyplot as plt
from shapely.geometry import Polygon
from shapely.geometry import box
from shapely.geometry import Point
import rasterio
import geopandas as gpd
import json
import requests
import urllib
import random
import pandas as pd
from google.colab import drive
# drive.mount('/content/drive')

In [11]:
'''one change'''

'one change'

In [76]:
%cd /content/drive/MyDrive/Proposal/figshare_Dijks

/content/drive/MyDrive/Proposal/figshare_Dijks


In [2]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is B4E0-BFE8

 Directory of c:\Users\zenus\Downloads\figshare_Dijks

04/20/2024  07:27 PM    <DIR>          .
04/20/2024  07:47 PM    <DIR>          ..
04/20/2024  07:17 PM    <DIR>          .ipynb_checkpoints
04/20/2024  07:14 PM             2,032 DEM.aux.xml
04/20/2024  07:14 PM    <DIR>          dembuff
04/20/2024  07:14 PM             2,038 dembuff.aux.xml
04/20/2024  07:14 PM                 5 Farm_point.cpg
04/20/2024  07:14 PM             1,425 Farm_point.dbf
04/20/2024  07:14 PM               403 Farm_point.prj
04/20/2024  07:14 PM               148 Farm_point.sbn
04/20/2024  07:14 PM               116 Farm_point.sbx
04/20/2024  07:14 PM               128 Farm_point.shp
04/20/2024  07:14 PM            24,096 Farm_point.shp.xml
04/20/2024  07:14 PM               108 Farm_point.shx
04/20/2024  07:14 PM               807 figshare_Dijks_readme.txt
04/20/2024  07:53 PM           392,126 figshare_Dijkstra.ipynb
04/20/2024  07:14 

In [4]:
from rasterio.plot import show
from rasterio.merge import merge
import rasterio as rio
from pathlib import Path
from osgeo import gdal
import numpy as np
import math
import shapely.geometry.point

In [5]:
# Weighted Average Interpolation module
# x and y are the unknown coordinates
# xCoords, yCoords, elevs are the known coordinates of the nehbor pixels
# order is the power of distance in the inverse distance that is used as a weight in IDW

def IDW(x, y, xCoords, yCoords, elevs, order=2):

    d =  np.zeros(len(xCoords))
    w =  np.zeros(len(xCoords))

    # if the distance from the unknown point to the central pixel is less than 1 cm, the evlevation of central pixel is assigned without using IDW
    if  math.sqrt((x-xCoords[0])**2 +(y-yCoords[0])**2) < 0.01:
        z = elevs[0]
    else:
        for i in range(len(xCoords)):
            d[i] = math.sqrt((x-xCoords[i])**2 +(y-yCoords[i])**2)
            w[i] = 1.0 / (d[i]**order)
        z = (np.sum(elevs*w)) / float(np.sum(w))

    return z

In [6]:
# This function get a 5*5 raster and return the m closest pixel to the point x,y located in the central pixel
def neibr(rasterBlock_x, rasterBlock_y, rasterBlock_elev, x, y, m):
    '''
    This function create the m closest pixel to the point(x,y)
    rasterBlock is the 5 by 5 matrix
    x,y is the location of sample points that needs to be interpolated
    '''
    if m ==4:
        if x <= rasterBlock_x[2,2]:
            if y >= rasterBlock_y[2,2]:
                xCoor = np.array([rasterBlock_x[2,2], rasterBlock_x [2,1], rasterBlock_x[1,1], rasterBlock_x[1,2]])
                yCoor = np.array([rasterBlock_y[2,2], rasterBlock_y [2,1], rasterBlock_y[1,1], rasterBlock_y[1,2]])
                elev = np.array([rasterBlock_elev[2,2], rasterBlock_elev [2,1], rasterBlock_elev[1,1], rasterBlock_elev[1,2]])
            elif y < rasterBlock_y[2,2]:
                xCoor = np.array([rasterBlock_x[2,2], rasterBlock_x [2,1], rasterBlock_x[3,1], rasterBlock_x[3,2]])
                yCoor = np.array([rasterBlock_y[2,2], rasterBlock_y [2,1], rasterBlock_y[3,1], rasterBlock_y[3,2]])
                elev = np.array([rasterBlock_elev[2,2], rasterBlock_elev [2,1], rasterBlock_elev[3,1], rasterBlock_elev[3,2]])
        elif x > rasterBlock_x[2,2]:
            if y >= rasterBlock_y[2,2]:
                xCoor = np.array([rasterBlock_x[2,2], rasterBlock_x [1,3], rasterBlock_x[1,2], rasterBlock_x[2,3]])
                yCoor = np.array([rasterBlock_y[2,2], rasterBlock_y [1,3], rasterBlock_y[1,2], rasterBlock_y[2,3]])
                elev = np.array([rasterBlock_elev[2,2], rasterBlock_elev [1,3], rasterBlock_elev[1,2], rasterBlock_elev[2,3]])
            elif y < rasterBlock_y[2,2]:
                xCoor = np.array([rasterBlock_x[2,2], rasterBlock_x [2,3], rasterBlock_x[3,2], rasterBlock_x[3,3]])
                yCoor = np.array([rasterBlock_y[2,2], rasterBlock_y [2,3], rasterBlock_y[3,2], rasterBlock_y[3,3]])
                elev = np.array([rasterBlock_elev[2,2], rasterBlock_elev [2,3], rasterBlock_elev[3,2], rasterBlock_elev[3,3]])
    return xCoor, yCoor, elev


In [7]:
def extractValue(x, y, dataset):
    '''
    This function extract value of a point from a raster
    :param x: x coordinate of point
    :param y: y coordinate of point
    :param dataset: a raster object imported by Rasterio
    :return: return the elevation of point
    '''
    val = dataset.sample([(x, y)])
    return list(val)[0][0]

def extractWindow(x, y, dataset, cellSize):
    '''
    This function aims to extract a 5*5 matrix from the input raster
    Also, this function convert the UTM coordinates to the local coordinate system of the extracted matrix
    :param x: x coordinate of point
    :param y: y coordinate of point
    :param dataset: a raster object imported by Rasterio
    :param cellSize: raster cell size
    :return: three 5*5 matrix (x, y, and elevation). Also the local coordinates of the input point
    '''
    row, col = dataset.index(x, y) # where the point is located in the raster
    rasterBlock_elev = dataset.read(1, window=((row-2, row+3), (col-2, col+3))) # extract the 5*5 raster block
    
    #find the upper left corner coordinates of the extracted 5*5 matrix
    # ulp = dataset.affine * (col-2, row-2)#src.affine * (col, row)
    ulp = dataset.transform * (col-2, row-2)
    ulpCX= ulp[0] + (cellSize/2.0) # coordinates of central point of the upper left corner pixel
    ulpCY= ulp[1] - (cellSize/2.0)

    # coordinates of the  5*5 matrix
    X = np.arange(ulpCX, ulpCX+cellSize*5, cellSize)
    Y = np.arange(ulpCY, ulpCY-cellSize*5, -cellSize)
    X, Y = np.meshgrid(X, Y)
    
    # A local coordinate system is created; the center of the 5*5 matrix is (0,0)
    x=x-X[2,2]
    y=y-Y[2,2]

    rasterBlock_x = X - X[2,2]
    rasterBlock_y = Y - Y[2,2]
    
    return x, y, rasterBlock_x, rasterBlock_y, rasterBlock_elev


In [8]:
from shapely.geometry import Point
import os
import geopandas as gpd
import pandas as pd
import rasterio
# Set the current workspace (all of the input DEMs are in this folder)

# Input data (DEMs): they are in ESRI Grid format
'''THIS ALWAYS NEEDS TO BE CHANGED IF YOU CHANGE A DEM'''
DEMs = ['dembuff']
resolutions = [10]


In [9]:
def eleva(dataframe):
    '''DO NOT FORGET TO CHANGE DEM HERE IF YOUR RE USING A DIFFERENT ONE
    I create a function out of this section since it gonna be used to extract 
    elevation from sampled points along our linestrings.
    
    dataframe = is the dataframe of points created , extracted by linestrings
    returns = a geodataframe with two new columns, elevation and interpolated elevation
    
    '''
    with rasterio.open('dembuff') as dem3m:
        samples = dataframe # randomPnts shapefile is imported as a geodataframe
        samples['elev3m'] = None # adding a new attribute "elev3m" to the geodataframe
        for index, row in samples.iterrows(): # Extract the elevation of each point from benchmark
            X = row['geometry'].x
            Y = row['geometry'].y
            samples.at[index, 'elev10m'] = extractValue(X,Y,dem3m).astype('float64')

    # Create the residual geodataframe. This dataframe is completed at the end
    residuals = samples.copy()
    '''Weighted average on extracted points of linestring'''
    # Methods used for calculating surface area
    #     methods = ['WP', 'WA4', 'Li3', 'BiLi4', 'BiQ9', 'BiC16']
    methods = ['WA4']

    #Add new feilds to the the "samples" geodataframe based on all combinations of resolutions and methods
    fields = []
    for res in resolutions:
        for mth in methods:
            samples[mth + str(int(res))] = None
            fields = fields + [mth + str(res)]

    for dem in DEMs:  # for each resolution surface-adjusted elevations are estimated for various interpolation methods
        with rasterio.open(dem) as src:
            cellSize = src.transform[0] # get actual DEM resolution fo computations
            res = resolutions[DEMs.index(dem)] # get nominal DEM resolution for labeling
            for index, row in samples.iterrows(): # iterating through all of the points
                X = row['geometry'].x
                Y = row['geometry'].y
                # The extractWindow function in the findValue module returns the 5*5 elevation matrix for the DEM based on the point
                # The coordinate of the central pixel is (0,0). x,y are the coordinates of the point in this local coordinate system
                x, y, rasterBlock_x, rasterBlock_y, rasterBlock_elev = extractWindow(X, Y, src, cellSize)

                try:
                    xCoor4, yCoor4, elev4 = neibr (rasterBlock_x, rasterBlock_y, rasterBlock_elev, x, y, 4)
                    samples.at[index, 'WA4'+ str(res)] = IDW(x, y, xCoor4, yCoor4, elev4, 2).astype('float64')

                except IndexError:
                    print ('index out of bounds'+str(index))
                    continue
    samples=gpd.GeoDataFrame(samples,geometry=samples['geometry'])
    return(samples)

In [10]:
'''Boulder roads and random points'''
import geopandas as gpd
import shapely
from shapely.geometry import LineString
from shapely.geometry import *
import networkx as nx   
import pandas as pd

ex = 'roads.shp'
extnd= gpd.read_file(ex)
simplify_extend = gpd.GeoDataFrame(extnd,geometry=extnd['geometry'])

rndm = gpd.read_file('rndm10m_pnts.shp')

farms_ = 'Farm_point.shp'
frms_ = gpd.read_file(farms_)
farm = gpd.GeoDataFrame(frms_, geometry=frms_['geometry'])

rnd_a = 'Power_plant.shp'
valmont = gpd.read_file(rnd_a)


In [11]:
rst_ = rasterio.open('/content/drive/MyDrive/Proposal/figshare_Dijks/dembuff')#you had top export another format for rasterio to work

r = gdal.Open("/content/drive/MyDrive/Proposal/figshare_Dijks/dembuff")
band = r.GetRasterBand(1) #bands start at one
a = band.ReadAsArray().astype(float)


def locate (xindex,yindex, raster):
  (upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = raster.GetGeoTransform()
  x_coords = xindex * x_size + upper_left_x + (x_size / 2) #add half the cell size
  y_coords = yindex * y_size + upper_left_y + (y_size / 2) #to centre the point
  return(x_coords,y_coords)

In [12]:
uni = simplify_extend.CID.unique() #This takes all the unique CID codes that are shared between line segments and 10m random pnts

In [41]:

'''For the first part the lines of the road network need to be combined with the 10 meter random samples'''
'''When running into runabouts you have to check if the first and last point of the linestrings are the same'''
indxe=[]
stand_line,stand_line1 = [] , []
folio_dist = []
folio_indx = []
PIX =[]
pnt = []
indxr,indxc = [],[]

for i in uni: 
  '''For each of the each one of CID codes, locate all the 10m randoms points associated with that segment,
  extract the first point of line segment, put the rest of random 10m points after the first point, 
  end with the last point of the line segment
  '''
 
  uo = np.where(rndm['CID'] == i ) #uo is the variable used to store all points of each unique CID
  ffs = 0.01
  # rndm_keep = rndm.loc[uo[0][0]:uo[0][-1]]
  # rndm1 = [i.coords[:][0] for i in rndm_keep.geometry]


  fuo = np.where(simplify_extend['CID'] == i ) #fuo is the variable that stores each line segment of road network
  rndm2 = simplify_extend.loc[fuo[0][0]:fuo[0][-1]] #rndm2 is the extraction of geometry that contains coords for each segment
  

  if rndm2.geometry[fuo[0][0]].coords[0] == rndm2.geometry[fuo[0][0]].coords[-1]: #quick check if there is a roundabout

    build = [] 
    build = [dd for dd in rndm2.geometry[fuo[0][0]].coords[:]]
    # build.append(rndm2[0])
    print('a circle was found ' , build)

  if rndm2.geometry[fuo[0][0]].coords[0] != rndm2.geometry[fuo[0][0]].coords[-1]:
  
    if len(uo[0])>0: # Segments of road that have random points associated to them, have a length larger than 0
      '''rndm2: the coordinates of road segments
         rndm1: the coordinates from all 10m random points associated to each unique CID segment
         build: the variable to store the combination of rndm2, rndm1
      '''
      
      rndm_keep = rndm.loc[uo[0][0]:uo[0][-1]]
      rndm1 = [k.coords[:][0] for k in rndm_keep.geometry]


      fuo = np.where(simplify_extend['CID'] == i )
      rndm2 = simplify_extend.loc[fuo[0][0]:fuo[0][-1]]
      
      # print(rndm2)
      build = []    
      rndm2 = [kk for kk in rndm2.geometry[fuo[0][0]].coords[:]]


      build.append(rndm2[0])
      build.extend(rndm1)
      build.append(rndm2[-1])

    
    if len(uo[0]) == 0: #In case segment does not have random points then we proceed with just the coordinates of road segment
      build = []

      fuo = np.where(simplify_extend['CID'] == i )
      rndm_keep = simplify_extend.loc[fuo[0][0]:fuo[0][-1]]
      rndm2 = [jj.coords[:] for jj in rndm_keep.geometry]
      ffs = 0.01
      
      build.append(rndm2[0][0])
      # build.extend(rndm1)
      build.append(rndm2[0][-1])

  p2p  = 0
  xc , yc = [],[]
  
  p2p_everyPix, pixel_only = [],[]
  new_line =[]
  new_line_dist = []

  pixel_only.append(0.0)


  for j in range(len(build)-1):
    '''This for loop uses the locate function to trace all points on road segment back to the raster, then also calculate pixel to pixel distance.
    rasterio is used to get the centroid coordinates and locate function to calculate distance.

    new_line: stores the geometry of all smaller lines that are created from incorporating 10m random points
    new_line_dist: stores the pixel to pixel distance
    folio_dist: pixel to pixel for each pair of points
    folio_indx: storing the CID codes
    folio: Geodataframe of new lines
    stand_line: These are the old segments that are now borken into small lines, after incorporating 10m random points in them
    folio1: Geodataframe of the initial line segments
    stand_line1: The old line segments that we can append pixel to pixel total lengths
    pnt: all points of all segments throughout the area
    indxr: the indexed row of the point
    indxc: the indexed column of a point
    '''


    rows,cols=rasterio.transform.rowcol(rst_.transform,build[j][0], build[j][1])
    rows1,cols1 = rasterio.transform.rowcol(rst_.transform,build[j+1][0], build[j+1][1])


    p2p+= math.sqrt((locate(rows,cols,r)[1] - locate(rows1,cols1,r)[1])**2 +(locate(rows,cols,r)[0] - locate(rows1,cols1,r)[0])**2)
    pixel_only.append(math.sqrt((locate(rows,cols,r)[1] - locate(rows1,cols1,r)[1])**2 +(locate(rows,cols,r)[0] - locate(rows1,cols1,r)[0])**2))

    # pnt.append(shapely.geometry.Point([locate(rows,cols,r)[1],locate(rows,cols,r)[0]]))

    xc.append(locate(rows,cols,r)[0])
    yc.append(locate(rows,cols,r)[1])

    p2p_everyPix.append(float(p2p))

    indxr.append(rows)
    indxc.append(cols)
    # indxe.append((rows,cols))

    '''Following to store the linestrings instead of the points '''
    # new_line.append(shapely.geometry.LineString([(locate(rows,cols,r)[0],locate(rows,cols,r)[1]),\
    #                                              (locate(rows1,cols1,r)[0],locate(rows1,cols1,r)[1])]))
    
    new_line.append(shapely.geometry.LineString([(build[j][0],build[j][1]),\
                                                 (build[j+1][0],build[j+1][1])]))
    
    new_line_dist.append(math.sqrt((locate(rows,cols,r)[1] - locate(rows1,cols1,r)[1])**2 +(locate(rows,cols,r)[0] - locate(rows1,cols1,r)[0])**2))

    folio_dist.append(math.sqrt((locate(rows,cols,r)[1] - locate(rows1,cols1,r)[1])**2 +(locate(rows,cols,r)[0] - locate(rows1,cols1,r)[0])**2))

    folio_indx.append(list(rndm_keep.CID)[0])
    
  pnt.extend(build)
  folio = gpd.GeoDataFrame(geometry = new_line, crs= rndm_keep.crs)
  folio['pix_pix'] = new_line_dist
  stand_line.append(folio)

  lifeline =  shapely.geometry.LineString(build)
  folio1 = gpd.GeoDataFrame(geometry = [lifeline], crs= rndm_keep.crs)
  folio1['pix_pix'] = p2p
  stand_line1.append(folio1)
  print(p2p)

  '''The following is an effort to save the highlighted points of the raster'''
  indxr.append(rows1)
  indxc.append(cols1)

  # indxe.append((rows1,cols1))

  # pnt.append(shapely.geometry.Point([locate(rows1,cols1,r)[1],locate(rows1,cols1,r)[0]]))
  # folio_indx.append(list(rndm_keep.CID)[0])
  # ind_x[f.Parcel_NO] = indxr
  
  # rndm.loc[uo[0][0]:uo[0][-1],'pix_dist'] = p2p_everyPix
  # rndm.loc[uo[0][0]:uo[0][-1],'pix_pix'] = pixel_only
  # rndm.loc[uo[0][0]:uo[0][-1],'indx_r'] = indxr
  # rndm.loc[uo[0][0]:uo[0][-1],'indx_c'] = indxc

  # xc.append(locate(rows1,cols1,r)[0])
  # yc.append(locate(rows1,cols1,r)[1])
  # pnt.append(shapely.geometry.Point([locate(rows1,cols1,r)[1],locate(rows1,cols1,r)[0]]))


682.2991780591568
1324.5226079254912
1963.4113644959832
738.3219783883596
18.174468393437564
17052.171962939243
1676.0739989992865
18.17446839355398
3102.664143844827
136.3085129512474
18.174468393495772
0.0
0.0
9.087234197184443
0.0
872.3744828877971
145.39574714750051
18.174468393495772
872.3744828877971
9.087234197184443
877.370430486109
18.174468393495772
1757.3327524232902
96.78160261467922
129.42621150492982
1826.8440003967498
1502.0279575674178
491.0889532752161
470.70955213288016
21.938524042549496
704.5052053002098
4793.830680408117
2654.1181953020364
5213.499548351836
875.7515798347612
580.3424445928401
29.40690758830299
1558.9268267005418
21.938524042549496
1524.782822663394
18.174468393961433
333.6955015796725
332.13637868169565
1952.1503331080141
1220.323697465668
1854.9817717941296
0.0
163.57021554093808
9.087234196718782
9.087234196253121
9.087234197184443
9.087234196718782
9.08723419677699
163.57021554093808
21.93852404235661
949.214718762531
1011.266235239587
1770.0784

In [44]:
pntt = [shapely.geometry.Point(i) for i in pnt]
pntgeo = gpd.GeoDataFrame(geometry = pntt, crs = simplify_extend.crs)
pntgeo['indxc'] = indxc
pntgeo['indxr'] = indxr
pntgeo.to_file('ALLOFPOINTS_withindexes') #Combine all points to new table with their indexed rows and columns

In [23]:
stand_lin1 = gpd.GeoDataFrame(geometry = pd.concat([stand_line1[d].geometry for d in range(len(stand_line1))],ignore_index=True),crs = rndm_keep.crs)
pixel_d = [stand_line1[d].pix_pix[0] for d in range(len(stand_line1))]
stand_lin1['pix_pix'] = pixel_d
simplify_extend['pix_pix'] = pixel_d
simplify_extend.to_file('initial_road_segments_withpixeldist') #Save the initial road segments with added pixel distances

In [20]:
stand_lin = gpd.GeoDataFrame(geometry = pd.concat([stand_line[d].geometry for d in range(len(stand_line))],ignore_index=True),crs = rndm_keep.crs)
stand_lin['pix_pix'] = folio_dist
stand_lin['CID'] = folio_indx #This is the file we need to proceed to next step, no need to store it now, we are going for surface adjusted next
stand_lin

,geometry,pix_pix,CID
0,"LINESTRING (485648.564 4418041.415, 485648.631...",9.087234,1
1,"LINESTRING (485648.631 4418041.522, 485654.061...",0.000000,1
2,"LINESTRING (485654.061 4418050.258, 485659.561...",12.851290,1
3,"LINESTRING (485659.561 4418059.106, 485665.265...",12.851290,1
4,"LINESTRING (485665.265 4418067.673, 485671.300...",9.087234,1
...,...,...,...
59209,"LINESTRING (455484.662 4431507.325, 455493.140...",9.087234,1025
59210,"LINESTRING (455493.140 4431502.015, 455501.626...",12.851290,1025
59211,"LINESTRING (455501.626 4431496.699, 455510.109...",9.087234,1025
59212,"LINESTRING (455510.109 4431491.386, 455518.587...",12.851290,1025


In [49]:
'''This cell deals with bringing the surface adjusted distance and combine it with the dataframe stand_lin
This cell has a long runtime, because we run through all the segments and all the random 10m sample points combined.'''
concat = []
crs = simplify_extend.crs
yo=0
concat1 = []
name = []
for i,g in stand_lin.iterrows():
    '''Stand_line has all data we need to pass through the surface adjusted part. Here we make use of function eleva, which for each 
    point of the road net, it extracts the interpolated elevation value based on IDW in a new column called WA410.

    tosave: the coordinates of each line connecting all 10m random points of a segment
    note: the dataframe that is passed to function eleva
    samples: the output of the function with the IDW stored in it
    d: is the surface adjusted distance that is calculated for the lines
    std: the standard deviation
    mean: mean of elevation
    saveL: the geodataframe that contains the new metrics
    concat: all segments of the road network
    its_adjusted: the final geodataframe that combines everything
    '''
    
    erasetosave=[]
    arraysave=[]
    if type(g['geometry'])!=shapely.geometry.MultiLineString:
        tosave=[]
        tosave.append(g['geometry'].coords[:])
        arraysave.append(tosave)
        yo+=1
   
        erasetosave.append(shapely.geometry.LineString(tosave[0]))
        
        for z in range(len(tosave)):
            passing=[]
            note=pd.DataFrame()
            note=pd.DataFrame(tosave[z],columns=['x','y'])
            # passing=[Point([tosave[z][kk][0],tosave[z][kk][1]]) for kk in range(len(tosave[z]))]
#             print passing
#             print note
            note['geometry']=gpd.GeoSeries([Point([tosave[z][kk][0],tosave[z][kk][1]]) for kk in range(len(tosave[z]))])
#             print note
            '''The extracted points that create the linestring are used to extract the elevation'''
            samples = eleva(note)
            print (samples)
        #     samples.to_file(driver = 'ESRI Shapefile', filename = output+ r'\samples'+str(i)+'.shp') #Export estimated eleavtions as a shapefile
            '''Calculating surface adjusted distance for the linestring'''
            try:
                d=0
                for ii in range(len(samples['geometry'])-1):
                    d += math.sqrt((samples['geometry'][ii].x-samples['geometry'][ii+1].x)**2 +(samples['geometry'][ii].y\
                                                    -samples['geometry'][ii+1].y)**2 + (samples['WA410'][ii]-samples['WA410'][ii+1])**2)
    #                 print d

                '''Euclidian distance between starting and ending point of linestring for comparisson'''
                # eud=0
                # eud += math.sqrt((tosave[z][0][0]-tosave[z][-1][0])**2 +(tosave[z][0][1]\
                #                                     -tosave[z][-1][1])**2)
    #                 print eud
                mean=0
                for yy in samples['WA410']:
                    mean+= yy
                mean = mean/samples.count()[0]
                # print (str('the mean is') + str(mean))
                
                dif=0
                for yy in samples['WA410']:
                    dif+= (yy-mean)**2
                std = dif/samples.count()[0]
                print ('standard is ' + str(std))
                
            except: #in case something is wrong , you ll be able to see it in the outfile
                print ('there was an elevation error')
                eud=-9999
                d=-9999
                std = -9999
            '''ta parakatw ta kanw comment in mono gia na kanw extract ta split roads pou den exoun name'''
            saveL=gpd.GeoDataFrame(geometry = [erasetosave[z]],crs = crs)
            # saveL['eud']=eud
            saveL['planar']=erasetosave[z].length
            saveL['surf_adj']=d
            saveL['std']=std
            saveL['mean']=mean
            saveL['geometry']=erasetosave[z]
            saveL['CID'] = g.CID
            if name == []:
                # saveL['name']=pd.core.series.Series(g['name']) #Becareful here the name = fixed 
                # saveL['eud']=eud
                saveL['planar']=erasetosave[z].length
                saveL['surf_adj']=d
                saveL['std']=std
                saveL['mean']=mean
                saveL['geometry']=erasetosave[z]

            crs=simplify_extend.crs
            saveL=gpd.GeoDataFrame(saveL, geometry=saveL['geometry'],crs=crs)
#             saveL.to_file(driver= 'ESRI Shapefile',filename = output+r'\saveL' +str(g['NAME'])+'.shp')
            concat.append(saveL)

its_adjusted=pd.concat([k for k in concat], ignore_index= True)

its_adjusted=gpd.GeoDataFrame(its_adjusted,geometry=its_adjusted['geometry'],crs=crs)   

# ontop.to_file('simplify_net_adjusted')

Streaming output truncated to the last 5000 lines.
               x             y                        geometry elev3m  \
0  481163.926773  4.432462e+06  POINT (481163.927 4432462.110)   None   
1  481163.911704  4.432472e+06  POINT (481163.912 4432472.118)   None   

       elev10m        WA410  
0  1597.873047  1597.864732  
1  1597.155762   1597.49458  
standard is 0.03425306132830121
               x             y                        geometry elev3m  \
0  481163.911704  4.432472e+06  POINT (481163.912 4432472.118)   None   
1  481163.896635  4.432482e+06  POINT (481163.897 4432482.127)   None   

       elev10m       WA410  
0  1597.155762  1597.49458  
1  1596.478882  1596.70641  
standard is 0.15530326928164345
               x             y                        geometry elev3m  \
0  481163.896635  4.432482e+06  POINT (481163.897 4432482.127)   None   
1  481163.881521  4.432492e+06  POINT (481163.882 4432492.165)   None   

       elev10m        WA410  
0  1596.478882   1

In [50]:
qdt = its_adjusted
qdt['pix_pix'] = stand_lin['pix_pix']
qdt

,geometry,planar,surf_adj,std,mean,CID,pix_pix
0,"LINESTRING (485648.564 4418041.415, 485648.631...",0.126698,5.068603e+36,6.422683e+72,-1.705164e+38,1,9.087234
1,"LINESTRING (485648.631 4418041.522, 485654.061...",10.285691,1.679821e+38,7.054501e+75,-8.399107e+37,1,0.000000
2,"LINESTRING (485654.061 4418050.258, 485659.561...",10.418212,1.049306e+01,3.912863e-01,1.754997e+03,1,12.851290
3,"LINESTRING (485659.561 4418059.106, 485665.265...",10.292189,1.036359e+01,3.687072e-01,1.753765e+03,1,12.851290
4,"LINESTRING (485665.265 4418067.673, 485671.300...",10.245017,1.027922e+01,1.754810e-01,1.752739e+03,1,9.087234
...,...,...,...,...,...,...,...
59209,"LINESTRING (455484.662 4431507.325, 455493.140...",10.003907,1.003026e+01,1.320087e-01,2.785528e+03,1025,9.087234
59210,"LINESTRING (455493.140 4431502.015, 455501.626...",10.012452,1.003525e+01,1.142642e-01,2.786229e+03,1025,12.851290
59211,"LINESTRING (455501.626 4431496.699, 455510.109...",10.009979,1.002171e+01,5.874014e-02,2.786810e+03,1025,9.087234
59212,"LINESTRING (455510.109 4431491.386, 455518.587...",10.004285,1.001613e+01,5.930028e-02,2.787296e+03,1025,12.851290


In [52]:
# qdt.to_file('its_adjusted_network')

In [ ]:
# indxe=[]
# stand_line,stand_line1 = [] , []
# folio_dist = []
# folio_indx = []
# PIX =[]
# pnt = []
# indxr,indxc = [],[]

# for i,g in its_adjusted.iterrows():
#   # rndm1 = [g.coords[:][0] for i in .geometry]
#   # print(g.geometry.coords[:][0])
#   rndm2 = g.geometry.coords[:]
#   if rndm2[0] == rndm2[-1]:
#       # print(rndm2)
#     build = []    
#     # ffs = (np.sum(rndm_keep.pix_pix))
#     # build = [dd for dd in rndm2.geometry[fuo[0][0]].coords[:]]
#     # build.append(rndm2[0])
#     print('a circle was found ' , rndm2)
#   # if rndm2[0] != rndm2[-1]:
#   p2p  = 0
  
#   xc , yc = [],[]
  
#   p2p_everyPix, pixel_only = [],[]
#   new_line =[]
#   new_line_dist = []

#   # rows,cols=rasterio.transform.rowcol(rst_.transform,short[0][0], short[0][1])
#   # fst = math.sqrt((locate(rows,cols,r)[1] - short[0][0])**2 +(locate(rows,cols,r)[0] - short[0][1])**2)


#   # p2p_everyPix.append(0.0)
#   pixel_only.append(0.0)
#   build= rndm2

#   for j in range(len(build)-1):


#     rows,cols=rasterio.transform.rowcol(rst_.transform,build[j][0], build[j][1])
#     rows1,cols1 = rasterio.transform.rowcol(rst_.transform,build[j+1][0], build[j+1][1])


#     # print(rows,cols , i)

#     # p2p+=math.sqrt((rasterio.transform.xy(rst_.transform,short[i][0],short[i][1])[0]-rasterio.transform.xy(rst_.transform,short[i+1][0],short[i+1][1])[0])**2\
#     #              +(rasterio.transform.xy(rst_.transform,short[i][0],short[i][1])[1]\
#     #                                               -rasterio.transform.xy(rst_.transform,short[i+1][0],short[i+1][1])[1])**2)
    

#     p2p+= math.sqrt((locate(rows,cols,r)[1] - locate(rows1,cols1,r)[1])**2 +(locate(rows,cols,r)[0] - locate(rows1,cols1,r)[0])**2)
#     pixel_only.append(math.sqrt((locate(rows,cols,r)[1] - locate(rows1,cols1,r)[1])**2 +(locate(rows,cols,r)[0] - locate(rows1,cols1,r)[0])**2))

#     pnt.append(shapely.geometry.Point([locate(rows,cols,r)[1],locate(rows,cols,r)[0]]))

   

#     xc.append(locate(rows,cols,r)[0])
#     yc.append(locate(rows,cols,r)[1])

#     p2p_everyPix.append(float(p2p))

#     indxr.append(rows)
#     indxc.append(cols)
#     # indxe.append((rows,cols))

#     '''Following to store the linestrings instead of the points '''
#     # new_line.append(shapely.geometry.LineString([(locate(rows,cols,r)[0],locate(rows,cols,r)[1]),\
#     #                                              (locate(rows1,cols1,r)[0],locate(rows1,cols1,r)[1])]))
    
#     new_line.append(shapely.geometry.LineString([(build[j][0],build[j][1]),\
#                                                  (build[j+1][0],build[j+1][1])]))
    
#     new_line_dist.append(math.sqrt((locate(rows,cols,r)[1] - locate(rows1,cols1,r)[1])**2 +(locate(rows,cols,r)[0] - locate(rows1,cols1,r)[0])**2))

#     folio_dist.append(math.sqrt((locate(rows,cols,r)[1] - locate(rows1,cols1,r)[1])**2 +(locate(rows,cols,r)[0] - locate(rows1,cols1,r)[0])**2))

#     # folio_indx.append(list(rndm_keep.CID)[0])
    
  
#   folio = gpd.GeoDataFrame(geometry = new_line, crs= its_adjusted.crs)
#   folio['pix_pix'] = new_line_dist
#   stand_line.append(folio)

#   lifeline =  shapely.geometry.LineString(build)
#   folio1 = gpd.GeoDataFrame(geometry = [lifeline], crs= its_adjusted.crs)
#   folio1['pix_pix'] = p2p
#   stand_line1.append(folio1)
#   print(p2p)

#   '''The following is an effort to save the highlighted points of the raster'''
#   indxr.append(rows1)
#   indxc.append(cols1)
#   indxe.append((rows1,cols1))
#   pnt.append(shapely.geometry.Point([locate(rows1,cols1,r)[1],locate(rows1,cols1,r)[0]]))
#   # folio_indx.append(list(rndm_keep.CID)[0])
#   # ind_x[f.Parcel_NO] = indxr
  
#   # rndm.loc[uo[0][0]:uo[0][-1],'pix_dist'] = p2p_everyPix
#   # rndm.loc[uo[0][0]:uo[0][-1],'pix_pix'] = pixel_only
#   # rndm.loc[uo[0][0]:uo[0][-1],'indx_r'] = indxr
#   # rndm.loc[uo[0][0]:uo[0][-1],'indx_c'] = indxc

#   xc.append(locate(rows1,cols1,r)[0])
#   yc.append(locate(rows1,cols1,r)[1])
#   # pnt.append(shapely.geometry.Point([locate(rows1,cols1,r)[1],locate(rows1,cols1,r)[0]]))

#   # index_dist_frompnts.append(p2p)

#   # print(len(uo[0]),uo[0])


In [ ]:
# stand_ = pd.concat([i['pix_pix'] for i in stand_line1])
# its_adjusted['pix_pix'] = list(stand_)

In [53]:
import networkx as nx 

In [54]:
'''Create three seperate graphs that contain the three different distance metrics'''
'''The term weight bellow specifies which distance each graph is going to be storing'''
newGraph = nx.Graph()
newGraph_adj = nx.Graph()
newGraph_pix = nx.Graph()
for i, elrow in its_adjusted.iterrows():
    newGraph.add_edge(elrow.geometry.coords[:][0], elrow.geometry.coords[:][-1],\
                      weight=elrow['planar'] ,attr_dict=elrow[2:].to_dict())
    

for i, elrow in its_adjusted.iterrows():
    newGraph_adj.add_edge(elrow.geometry.coords[:][0], elrow.geometry.coords[:][-1],\
                          weight=elrow['surf_adj'] ,attr_dict=elrow[2:].to_dict())
    
    
for i, elrow in its_adjusted.iterrows():
    newGraph_pix.add_edge(elrow.geometry.coords[:][0], elrow.geometry.coords[:][-1], weight = elrow['pix_pix'],\
                         attr_dict=elrow[1:].to_dict())

In [55]:
'''From the graphs you need to capture the nodes that are going to be used for Djikstra routine'''
'''Three seperate variables are created in order to make sure that all nodes are captured from each graph, although one set could also be used'''
checkNodes = [shapely.geometry.Point(i) for i in list(newGraph.nodes)]
checknodes = gpd.GeoDataFrame(geometry=checkNodes , crs=crs)

checkNodes_adj = [shapely.geometry.Point(i) for i in list(newGraph_adj.nodes)]
checknodes_adj=gpd.GeoDataFrame(geometry=checkNodes_adj , crs=crs)

checkNodes_pix = [shapely.geometry.Point(i) for i in list(newGraph_pix.nodes)]
checknodes_pix = gpd.GeoDataFrame(geometry=checkNodes_pix , crs=its_adjusted.crs)

In [56]:
end = (valmont['geometry'][0].x,valmont['geometry'][0].y)

In [57]:
'''Since Valmont point is not part of the nodes created from the graphs, the point is replaced with the closest node from the graphs above'''
'''Create a buffer with radius 10m in this case and find out which node is the closest to the end point, Valmont power plant'''
buffclosest = valmont['geometry'][0].buffer(10)
for i in range(len(checkNodes)):
    if buffclosest.contains(checkNodes[i])==True:
        print(i,checkNodes[i])
        print(shapely.geometry.LineString([valmont['geometry'][0],checkNodes[i]]).length, 'meters away')

44220 POINT (482536.9678157959 4429399.89875371)
1.8360364311696464 meters away
44221 POINT (482546.9714681292 4429400.116803702)
9.512630020550585 meters away


In [58]:
'''The above showed that node 44221 is 9 meters away, here we replace valmont with the Node that belongs to the graph'''
end = (checkNodes[44221].x,checkNodes[44221].y) 

In [59]:
end

(482546.9714681292, 4429400.116803702)

In [60]:
'''As above, instead of having the farm itself as a starting point, we use the farm
and create an increasing buffer until we find the closest node to that farm. Then we use that node as the start'''
buffname=[]
buffcontainn = []
starting = []
for i,j in farm.iterrows():
    buffcontain=[]
    buffsize=10
    while buffcontain == []:

    #     buffclosest = thepoint_inside[indx].buffer(buffsize)
        buffclosest = j['geometry'].centroid.buffer(buffsize)
        
        for i in range(len(checkNodes)):
            if buffclosest.contains(checkNodes[i])==True:
                buffcontain.append(checkNodes[i])
        buffsize+=10
    buffcontainn.append(buffcontain[0])
    #     print 'buffer' , buffsize, 'contains', len(buffcontain)

In [61]:

buffname1=[]
buffcontainn1 = []
starting = []
for i,j in farm.iterrows():
    buffcontain=[]
    buffsize=10
    while buffcontain == []:

    #     buffclosest = thepoint_inside[indx].buffer(buffsize)
        buffclosest = j['geometry'].centroid.buffer(buffsize)
        
        for i in range(len(checkNodes_pix)):
            if buffclosest.contains(checkNodes_pix[i])==True:
                buffcontain.append(checkNodes_pix[i])
        buffsize+=10
    buffcontainn1.append(buffcontain[0])


In [62]:
'''Finally run Dijkstra's routine using networkx library, for three different distance metrics'''
how_lin =[]
for l,k in enumerate(buffcontainn):
    '''Three routines initiated for different graphs, newGraph, newGraph_adj, newGraph_pix
    this is being done just in case one of these routes will take a different path, thats why we need three.

    aloo: the output geodataframe which contains the route and distance metrics for planar
    aloo_ad: the output geodataframe which contains the route and distance metrics for adjusted
    aloo_ds: the output geodataframe which contains the route and distance metrics for pixel to pixel
    sequence: stores the sequence segments from the start point to end point
    how_lin: combines the three geodataframes 
    '''
    
    strt = (k.x,k.y)
    showme= nx.dijkstra_path(newGraph,strt,end)
    showmee=[shapely.geometry.LineString([([showme[i][0],showme[i][1]]), \
                                 (showme[i+1][0],showme[i+1][1])]) for i in range(len(showme)-1)]

    letsee = shapely.geometry.MultiLineString([i for i in showmee])
    p2pD = letsee.length


    alo=[shapely.geometry.Point([i[0],i[1]]) for i in showme]
    aloo = [shapely.geometry.LineString(alo)] 
    
    
    showme_ad= nx.dijkstra_path(newGraph_adj,strt,end)
    showmee_ad=[shapely.geometry.LineString([([showme_ad[i][0],showme_ad[i][1]]), \
                                 (showme_ad[i+1][0],showme_ad[i+1][1])]) for i in range(len(showme_ad)-1)]

    letsee = shapely.geometry.MultiLineString([i for i in showmee_ad])
    p2pD = letsee.length
    adj_len = nx.dijkstra_path_length(newGraph_adj,strt,end)

    alo_ad=[shapely.geometry.Point([i[0],i[1]]) for i in showme_ad]
    aloo_ad = [shapely.geometry.LineString(alo_ad)] 

    aloo_ad = gpd.GeoDataFrame(geometry = aloo_ad,crs=crs)
    aloo_ad['vector'] = p2pD
    aloo_ad['surface_adjusted'] = adj_len
    aloo_ad['diff_distances'] = adj_len - p2pD
#     aloo_ad['pix_pix'] = dis_pix
    
    aloo = gpd.GeoDataFrame(geometry = aloo,crs=crs)
    aloo['vector'] = p2pD
    aloo['surface_adjusted'] = adj_len
    aloo['diff_distances'] = adj_len - p2pD
#     aloo['pix_pix'] = dis_pix

    
    sequence = gpd.GeoDataFrame(geometry = showmee,crs = its_adjusted.crs)
#     sequence.to_file('sequence'+str(buffname[l])) #this to catch the sequence of all smaller segments from-to
    
    strt1 = (buffcontainn1[l].x,buffcontainn1[l].y)
    forshow = nx.dijkstra_path(newGraph_pix,strt1,\
                          end)

    forshoww=[shapely.geometry.LineString([([forshow[i][0],forshow[i][1]]), \
                                 (forshow[i+1][0],forshow[i+1][1])]) for i in range(len(forshow)-1)]
    forsh = shapely.geometry.MultiLineString([i for i in forshoww])

    dis_pix = nx.dijkstra_path_length(newGraph_pix,strt1,end)

    alo_ds=[shapely.geometry.Point([i[0],i[1]]) for i in forshow]
    aloo_ds = [shapely.geometry.LineString(alo_ds)] 
    
    aloo_ds = gpd.GeoDataFrame(geometry = aloo_ds,crs=crs)
    aloo_ds['vector'] = p2pD
    aloo_ds['surface_adjusted'] = adj_len
    aloo_ds['diff_distances'] = adj_len - p2pD
    aloo_ds['pix_pix'] = dis_pix

    how_lin.append(aloo)
    how_lin.append(aloo_ad)
    how_lin.append(aloo_ds)
    
    print(l)   

0


In [64]:
'''Calculating the error between the distances planar and surface adjusted, the equation is 
((surface adjusted - planar)/surfahce adjusted) * 100'''
how_lin[2]['surfplanar_err'] = ((how_lin[2].surface_adjusted[0] - how_lin[2].vector[0])/how_lin[2].surface_adjusted[0]) * 100

In [65]:
how_lin[2]

,geometry,vector,surface_adjusted,diff_distances,pix_pix,surfplanar_err
0,"LINESTRING (453860.850 4422080.967, 453868.944...",35868.433311,36013.943169,145.509858,37952.847801,0.404038


In [70]:
'''In case the route changes between planar or adjusted or pixel to pixel, this geodataframe will capture that.
Since for this route it doesnt change, it is not saved as an extra output at this point.''' 
how_ = pd.concat([k for k in how_lin], ignore_index = True)
how_

,geometry,vector,surface_adjusted,diff_distances,pix_pix,surfplanar_err
0,"LINESTRING (453860.850 4422080.967, 453868.944...",35868.433311,36013.943169,145.509858,NaN,NaN
1,"LINESTRING (453860.850 4422080.967, 453868.944...",35868.433311,36013.943169,145.509858,NaN,NaN
2,"LINESTRING (453860.850 4422080.967, 453868.944...",35868.433311,36013.943169,145.509858,37952.847801,0.404038


In [ ]:
how_lin[2].to_file('test_route') #final output of Dijkstra routine
sequence.to_file('sequence') #final output for the sequence from start point to end point